# Staż Techmo - zadania

Poniżej znajdują się 3 zadania, które należy rozwiązać poprzez napisanie odpowiedniego kodu (w komórkach pod treścią zadania w pliku **staz.ipynb**), pozwalającego na wykonanie wymaganych w zadaniu obliczeń i prezentacje wyników. Przy rozwiązywaniu zadań można korzystać z dowolnych paczek dostępnych w managerze Pip np. numpy, pandas, librosa itp.

Notebook z rozwiązaniem oraz dane wrzuć na swojego githuba razem z plikiem *requirements.txt* pozwalającym na łatwą instalacje wszystkich wymaganych paczek.

## 1. Przetwarzanie sygnałów

* Wczytaj plik dźwiękowy *audio.wav*
* Wstaw widget z audioplayerem tak, aby można było odsłuchać plik
* Narysuj **oscylogram**
* Wyznacz i narysuj przebieg tonu podstawowego **F0**
* Wyznacz i narysuj w skali decybelowej krótkoczasowe widmo sygnału (**STFT**). Dobierz odpowiednie wg Ciebie parametry analizy. Uzasadnij swój wybór.
* Wyznacz i narysuj w skali decybelowej wartości **MFCC** (13 współczynników). Parametry analizy pozostaw takie jak w kroku poprzednim.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as sig
import tkinter


from python_speech_features import mfcc
from scipy.io import wavfile
from playsound import playsound

sample_rate, data = wavfile.read("audio.wav")

#Play widget
top = tkinter.Tk()
def playaudio():
   playsound('audio.wav')

B = tkinter.Button(top, text ="Play", command = playaudio)

B.pack()
top.mainloop()

amount_of_samples = len(data)

length_of_sound = amount_of_samples / sample_rate

scaled_data = data / (2.**15)

time_array = np.arange(0, float(amount_of_samples), 1)/ sample_rate
plt.plot(time_array, scaled_data, linewidth=0.3, alpha=0.7, color='#004bc6')
plt.xlabel('Time(s)')
plt.ylabel('Amplitude')
plt.show()


spectrum, freqs, t, im = plt.specgram(data, Fs=sample_rate, NFFT=4096, cmap=plt.get_cmap('coolwarm'))
cbar = plt.colorbar(im)
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)')
plt.title('SFTP')
cbar.set_label('Intensity (dB)')
plt.show()



mfcc_feat = mfcc(data, sample_rate)

ig, ax = plt.subplots()
mfcc_data= np.swapaxes(mfcc_feat, 0 ,1)
cax = ax.imshow(mfcc_data, interpolation='nearest', cmap=plt.get_cmap('coolwarm'), origin='lower',
                aspect='auto')
dbar = plt.colorbar(cax)
ax.set_title('MFCC')
#Showing mfcc_data
dbar.set_label('Intensity (dB)')
plt.show()
#Showing mfcc_feat
plt.plot(mfcc_feat)
#plt.show()


## 2. Metryki

W pliku *predicted.json* zapisane są wyniki rozpoznania systemu ASR, służącego do rozpoznawania cyfr.

Dla każdej cyfry od 0 do 9 w słowniku przedstawione są wartości, jakie system rozpoznał dla kolejnych nagrań danej cyfry. 

Oblicz i przedstaw w formie tabeli **macierz pomyłek** (confusion matrix) oraz oblicz **skuteczność** systemu.

In [ ]:
import json


with open("predicted.json", "r") as file:
    data = json.load(file)

#creating confusion matrix 10x10
conf_matrix = [[0] * 10 for i in range(10)]
count_all=0
count_tr=0


for i in range(len(data)):
    for j in range(len(data[str(i)])):
        a=data[str(i)][j]
        conf_matrix[i][int(a)] += 1
    count_tr += conf_matrix[i][i]

count_all = (i+1)*(j+1)
efficiency = count_tr/count_all*100

print("Efficiency is: ")
print(efficiency,"%")

#output matrix
print("Confusion matrix:")
for i in range(len(data)):
    print(conf_matrix[i])


## 3. Algebra

W plikach tekstowych `matrix_P.txt` oraz `matrix_Q.txt` znajduje się po 8 pomiarów 4-wymiarowej zmiennej (8 wierszy po 4 wartości).
* Wczytaj macierze do osobnych zmiennych `P` , `Q`.
* Korzystając z biblioteki `numpy` zaimplementuj funkcję obliczającą wzór:

$$
D_{KL}(P, Q) = \frac{1}{2} \mathrm{tr}\{(\boldsymbol{\Sigma}_p^{-1} + \boldsymbol{\Sigma}_q^{-1})(\boldsymbol{\mu}_p - \boldsymbol{\mu}_q)(\boldsymbol{\mu}_p - \boldsymbol{\mu}_q)^T + \boldsymbol{\Sigma}_p \boldsymbol{\Sigma}_q^{-1} + \boldsymbol{\Sigma}_q \boldsymbol{\Sigma}_p^{-1} - 2 \boldsymbol{I}\}
$$

gdzie:

$\mathrm{tr}$ – ślad macierzy (trace)

$\boldsymbol{\Sigma}$ – macierz kowariancji

$\boldsymbol{\mu}$ – uśredniony wektor ze wszystkich pomiarów

$\boldsymbol{I}$ – macierz jednostkowa

> Dla podanych danych funkcja powinna zwrócić wartość ok. 6.03

In [ ]:
import numpy as np

#read matrixs
P = np.genfromtxt("matrix_P.txt")
Q = np.genfromtxt("matrix_Q.txt")

#create covarianty matrix

cov_P=np.cov(P.T)
cov_Q=np.cov(Q.T)
#create average variable
average_P = np.array([np.mean(P[:,0]),np.mean(P[:,1]),np.mean(P[:,2]),np.mean(P[:,3])])
average_Q = np.array([np.mean(Q[:,0]),np.mean(Q[:,1]),np.mean(Q[:,2]),np.mean(Q[:,3])])

aver = average_P - average_Q
    
inv_P = np.linalg.inv(cov_P)
inv_Q = np.linalg.inv(cov_Q)

first= np.add(inv_P,inv_Q)
#reshape instead of tranform because np cant T on 1D
pom= aver*aver.reshape(4,1)

a=first*pom

dod=cov_P*inv_Q
dod2=cov_Q*inv_P

#identity matrix
b=2*np.identity(4)

c=a+dod+dod2-b
#trace of matrix
d=0.5*c.diagonal().sum()

print(d)
